In [35]:
from flask import Flask
import datetime
import json
import joblib
from scipy.spatial.distance import cdist
from math import radians, cos, sin, asin, sqrt

In [37]:
app = Flask(__name__)

@app.route('/avoid-points', methods=['POST'])
def my_endpoint():
    data = request.json
    g_LAT_LON = data['myLocation']
    
    ### my code for file
    def load_json_file(file_path1, file_path2, file_path3):
        with open(file_path1, 'r') as file1:
            police_coordinate = json.load(file1)
        with open(file_path2, 'r') as file2:
            hospital_coordinate = json.load(file2)
        with open(file_path3, 'r') as file3:
            crime_locations = json.load(file3)
        return police_coordinate, hospital_coordinate, crime_locations

    ### my code for year, month, day, time
    def current_datetime():
        current_time = datetime.datetime.now()
        year = current_time.year
        month = current_time.month
        day = current_time.day
        hour = current_time.hour
        def categorize_time(hour):
            categories = None
            if hour >= 0 and hour < 3:
                categories = 4
            elif hour >= 3 and hour < 6:
                categories = 1
            elif hour >= 6 and hour < 9:
                categories = 6
            elif hour >= 9 and hour < 12:
                categories = 4
            elif hour >= 12 and hour < 15:
                categories = 0
            elif hour >= 15 and hour < 18:
                categories = 3
            elif hour >= 18 and hour < 21:
                categories = 2
            else:
                categories = 5
            return categories

        return year, month, day, categorize_time(hour)
    
    ### calculate distance to the closest police office/hospital
    def calculate_hospital_policeman(p_loc, h_loc):
        # calculate distance matrix using cdist
        police_distances = cdist(g_LAT_LON, p_loc)
        hospital_distances = cdist(g_LAT_LON, h_loc)
        # get minimum distance for each row in A
        p_min_distances = police_distances.min(axis=1)
        h_min_distances = hospital_distances.min(axis=1)
        return p_min_distances, h_min_distances
    
    def dangerous_level_check():
        dangerous_level = []
        for i in range(len(g_LAT_LON)):
            lat_start, lat_end, lon_start, lon_end, step = 33.706, 34.335, -118.668, -118.155, 0.0015
            current_Lat = g_LAT_LON[i][[0]]
            current_Lon = g_LAT_LON[i][[1]]
            index_Lat = int((current_Lat - lat_start) / step)
            index_Lon = int((current_Lon - lon_start) / step)
            safety_value = regions[index_Lat][index_Lon]
            if safety_value >= 1600:
                dangerous_level.append(1)
            elif safety_value >= 530:
                dangerous_level.append(0)
            elif safety_value >= 40:
                dangerous_level.append(2)
            else: 
                dangerous_level.append(3)
        return dangerous_level
    
    police, hospital, regions = load_json_file("police_coordinates.json", "hospital_coordinates.json", "location.json")
    p_list, h_list = calculate_hospital_policeman(police, hospital)
    year, month, day, time = current_datetime()
    danger_list = dangerous_level_check()
    
    def make_input_for_model():
        output = []
        for i in range(len(g_LAT_LON)):
            each_data = [g_LAT_LON[i][0], g_LAT_LON[i][1], day, month, year, p_list[i], h_list[i], danger_list[i], time]
            output.append(each_data)
        return output
    
    predict_x = make_input_for_model()
    ### 
    model = joblib.load("crime_model.joblib")
    # do something with the array
    prediction = model.predict(predict_x)
    problem_point = []
    for i in range(len(prediction)):
        if prediction[i] == "Assaults and threats":
            problem_point.append(g_LAT_LON[i])
    ### return my predictions
    return jsonify({'problem_points': problem_point})

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Apr/2023 15:27:41] "GET / HTTP/1.1" 404 -
